Notebook com três tipos de data augumentation: espelhamento horizontal, zoom e rotação de 90 graus.

Caminhos de entrada e saída

In [1]:
input_base_dir = r'E:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\mamoeiro\masks'
output_base_dir = r'E:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\mamoeiro\augumented\masks'

Bibliotecas necessárias

In [2]:
import os
import json
import cv2
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from tqdm import tqdm

Criação das pastas de saída para train, valid, test

In [3]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

Função para aplicar zoom

In [4]:
def zoom_image(image, zoom_factor):
    height, width = image.shape[:2]
    new_width, new_height = int(width * zoom_factor), int(height * zoom_factor)
    image_zoomed = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
    
    if zoom_factor > 1:
        # Recortar a imagem centralizada para o tamanho original
        start_x = (new_width - width) // 2
        start_y = (new_height - height) // 2
        return image_zoomed[start_y:start_y+height, start_x:start_x+width]
    else:
        # Preencher com bordas se o zoom for menor que 1
        border_x = (width - new_width) // 2
        border_y = (height - new_height) // 2
        return cv2.copyMakeBorder(image_zoomed, border_y, border_y, border_x, border_x, cv2.BORDER_CONSTANT, value=[0, 0, 0])

Função para aplicar rotação e ajustar os labels

In [5]:
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

Função para aplicar espelhamento e ajustar os labels

In [6]:
def mirror_image(image):
    return cv2.flip(image, 1)

Função principal para processar as imagens e aplicar data augmentation

In [7]:
def augment_data(input_base_dir, output_base_dir):
    # Criar diretório de saída, se necessário
    create_dir(output_base_dir)
    
    subfolders = ['Train', 'Val', 'Test']  # Pastas que devem ser varridas

    for subfolder in subfolders:
        input_dir = os.path.join(input_base_dir, subfolder)
        output_dir = os.path.join(output_base_dir, subfolder)
        create_dir(output_dir)

        for root, _, files in os.walk(input_dir):
            for file in files:
                if file.endswith('.png') or file.endswith('.jpg'):
                    img_path = os.path.join(root, file)
                    img = cv2.imread(img_path)

                    # Caminhos para salvar as imagens transformadas
                    base_name, ext = os.path.splitext(file)
                    img_shape = img.shape

                    # Aplicar espelhamento
                    mirrored_img = mirror_image(img)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_mirror' + ext), mirrored_img)

                    # Aplicar rotação
                    rotated_img = rotate_image(img, 90)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_rotate' + ext), rotated_img)

                    # Aplicar zoom
                    zoomed_img = zoom_image(img, 1.2)
                    cv2.imwrite(os.path.join(output_dir, base_name + '_zoom' + ext), zoomed_img)

augment_data(input_base_dir, output_base_dir)